In [1]:
from dotenv import load_dotenv
import os
load_dotenv(dotenv_path=".env", override=True)

True

In [2]:
## Upload and Split the Doc
import chromadb
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

current_directory = os.getcwd()
print(f"Current directory: {current_directory}")
os.chdir("..")
os.chdir("..")
current_directory = os.getcwd()
print(f"Current directory: {current_directory}")

chroma_client = chromadb.Client()
pdf_loader = PyPDFLoader("./documents/Employee_Leave_Policy.pdf")
pages = pdf_loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=500)
doc_splits = text_splitter.split_documents(pages)
print("Total number of documents: ",len(doc_splits))

Current directory: /Users/saha/Documents/Projects/ai-agents/multi_agent_langgraph/src/agents
Current directory: /Users/saha/Documents/Projects/ai-agents/multi_agent_langgraph
Total number of documents:  13


In [3]:
## Create embeddings and save to vector store
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path=".env", override=True)
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
#llm = ChatOpenAI(model="o4-mini", temperature=0.7)
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

vector_store = Chroma.from_documents(
    documents=doc_splits,
    collection_name="policy_collection",
    embedding=embeddings,
    persist_directory="./chroma_policy_db",  # Where to save data locally, remove if not necessary
)


In [5]:
## Tools

from langchain_openai import ChatOpenAI
from typing import List, Dict , TypedDict
from typing import Annotated, Literal, TypedDict
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain.schema import Document
from pydantic import BaseModel,Field
import openai
from langsmith.wrappers import wrap_openai
from langsmith import traceable
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent
from typing import List, Dict , TypedDict
from typing import Annotated, Literal, TypedDict
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langgraph.graph import MessagesState, StateGraph
from langchain_core.tools import tool
import requests

llm = ChatOpenAI(model="gpt-4o")

# Create specialized agents

@tool
def generate_ans_from_policy(query: str):
    """
    Retrieve information related to a query.
    Input Parameters:
        query (str) : This is an user question
    Returns:
        str: Answer to the user question.
    """
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
            (f"Source: {doc.metadata}\n" f"Content: {doc.page_content}")
            for doc in retrieved_docs
        )

    llm = ChatOpenAI(model="gpt-4o", temperature=0)
    prompt_fmt = f"""You are an assistant for question-answering tasks.
                    Use the following pieces of retrieved context to answer the question.
                    If you don't know the answer, say that you don't know.
                    Your answer must be very concise. 
                    Context: {serialized}.
                    Question: {query}.
                """
    #print("Prompt = ",prompt_fmt)
    llm_response = llm.invoke(prompt_fmt)
    return {"messages": [llm_response]}


class Employee(BaseModel):
    """Represents a single employee with name, age, and salary."""
    full_name: str = Field(description="The full name of the employee.")
    basic_salary: int = Field(description="The yearly base salary of the employee.")
    earn_leaves: float = Field(description="The total accumulated earn leaves of the employee.")
    sick_leaves: float = Field(description="The total accumulated sick leaves the employee.")



@tool
def fetch_emp_data(username:str, password :str) -> dict:
    """
    Fetches employee data from an external API.

    Parameters:
        username (str) : An username of an employee. This can be an email id as well
        password (str) : A passphrase used to authenticate an employee

    Returns:
        dict: A dictionary with employee details.
    """
    payload = {
        "username": username.lower(),
        "password": password
    }
    base_url = "http://127.0.0.1:5000/find_details"
    
    print(f"--- Calling external API to fetch leave data for username: {username} ---")
    
    try:
        response = requests.post(base_url,data=payload)
        data = response.json()
        return data
    except requests.exceptions.ConnectionError as e:
        print(f"\nError: Could not connect to the Flask server.")
        print(f"Error details: {e}")
    except requests.exceptions.HTTPError as e:
        print(f"\nError: HTTP request failed with status code {response.status_code}")
        print(f"Error details: {e}")
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")



@tool
def generate_email(no_of_leaves: int,total_enchashment: float, purpose: str) -> str:
    """
   Returns an email with the following information about the employee 
        1. No of leaves
        2. Total encashment amount
        2. Purpose of the email
    The target recepient of the email is an HR Manager of the department

    Parameters:
    no_of_leaves (int): No of leaves 
    total_encashment (float) : Total value of the leaves
    purpose (str): Purpose of the email

    Returns:
    str :  This is an email generated
    
    """

    llm = ChatOpenAI(model="gpt-4o", temperature=0)
    prompt_fmt = f"""You are email writing assistant.
                    Please write a short email to a HR manager with the following information.
                    Leaves : {no_of_leaves}
                    Total Encashment value : {total_enchashment}
                    Purpose : {purpose}
                    Please include the date on which the email is written.
                """
    llm_response = llm.invoke(prompt_fmt) 
    return (llm_response)
    




In [6]:
## Agents

policy_agent = create_react_agent(
    model=llm,
    tools=[generate_ans_from_policy],
    name="policy_agent",
    prompt="You are an agent that answers to uemployeeser questions on company leave policy"
)


employee_agent = create_react_agent(
    model=llm,
    tools=[fetch_emp_data],
    response_format=Employee,
    name="employee_agent",
    prompt="You are an agent that pulls employee information from an external API"
)

email_agent = create_react_agent(
    model=llm,
    tools=[generate_email],
    name="email_agent",
    prompt="You are an agent that writes an email to a medical practitioner"
)

# Create supervisor workflow
workflow = create_supervisor(
    #[employee_agent],
    [policy_agent,employee_agent,email_agent],
    model=llm,
    prompt=(
        "You are a leave management system respoding to employee queries. "
        "For fetching employee details use employee_agent. "
        "For answering questions related to company leave policy use policy_agent."
        "To write the email to the HR supervisor use the email_agent"
    )
)



In [6]:
# # Compile and run
# app = workflow.compile()
# result = app.invoke({
#     "messages": [
#         {
#             "role": "user",
#             "content": "How many sick leaves is an employee entitiled to as per the policy? Once you have the information help me to generate an email that I need to send to my HR manager"
#         }
#     ]
# })

# print(result)

In [ ]:
# app = workflow.compile()
# result = app.invoke({
#     "messages": [
#         {
#             "role": "user",
#             "content": "Find formulae to calculate the leave encashment amount for an employee as per the policy.\
#                         Use this formula to calculate the leave encashment amount for the remaining earn leaves for the employee with username = risaha@redhat.com and password = Hello!123 \
#                         Once you all the information, Write an email to be send as a leave encashment request to the HR manager with all the details "
                        
#         }
#     ]
# })

# print(result)

In [7]:
app = workflow.compile()
result = app.invoke({
    "messages": [
        {
            "role": "user",
            "content": "Fetch the employee details like earn leaves and salary for the employee with username = risaha@redhat.com and password = Hello!123 \
                        Fetch the formule to calculate the leave encashment amount from the employee leave policy document. Do not assume.\
                        Use this formula to calculate the leave encashment amount for the employee \
                        Once you all the information, Write an email to be send as a leave encashment request to the HR manager with all the details "
                        
        }
    ]
})

print(result)

--- Calling external API to fetch leave data for username: risaha@redhat.com ---
--- Calling external API to fetch leave data for username: risaha@redhat.com ---
{'messages': [HumanMessage(content='Fetch the employee details like earn leaves and salary for the employee with username = risaha@redhat.com and password = Hello!123                         Fetch the formule to calculate the leave encashment amount from the employee leave policy document. Do not assume.                        Use this formula to calculate the leave encashment amount for the employee                         Once you all the information, Write an email to be send as a leave encashment request to the HR manager with all the details ', additional_kwargs={}, response_metadata={}, id='8e59d1d6-c2f8-41ef-a1e4-7206cebfc272'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_vtnVKr3JCNGbf2rHtG0v7clR', 'function': {'arguments': '{}', 'name': 'transfer_to_employee_agent'}, 'type': 'function'}], 'refu

In [8]:
for msg in result['messages']:    
    msg.pretty_print()

================================ Human Message =================================

Fetch the employee details like earn leaves and salary for the employee with username = risaha@redhat.com and password = Hello!123                         Fetch the formule to calculate the leave encashment amount from the employee leave policy document. Do not assume.                        Use this formula to calculate the leave encashment amount for the employee                         Once you all the information, Write an email to be send as a leave encashment request to the HR manager with all the details 
================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  transfer_to_employee_agent (call_vtnVKr3JCNGbf2rHtG0v7clR)
 Call ID: call_vtnVKr3JCNGbf2rHtG0v7clR
  Args:
================================= Tool Message =================================
Name: transfer_to_employee_agent

Successfully transferred to employee_agent
=======================